In [1]:
import pandas as pd

df = pd.read_excel("/Users/khuyentran/Downloads/distance.xlsx", header=2, index_col=0)
df

,1,2,3,4,5,6,7,8
1,0,3,4,6,8,9,8,10
2,3,0,5,4,8,6,12,9
3,4,5,0,2,2,3,5,7
4,6,4,2,0,3,2,5,4
5,8,8,2,3,0,2,2,4
6,9,6,3,2,2,0,3,2
7,8,12,5,5,2,3,0,2
8,10,9,7,4,4,2,2,0


In [2]:
df = df[df <=2]
df

,1,2,3,4,5,6,7,8
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0.0,2.0,2.0,NaN,NaN,NaN
4,NaN,NaN,2.0,0.0,NaN,2.0,NaN,NaN
5,NaN,NaN,2.0,NaN,0.0,2.0,2.0,NaN
6,NaN,NaN,NaN,2.0,2.0,0.0,NaN,2.0
7,NaN,NaN,NaN,NaN,2.0,NaN,0.0,2.0
8,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0


In [3]:
df.replace([0,2],[1,1],inplace=True)

In [4]:
df.fillna(0,inplace=True)
df

,1,2,3,4,5,6,7,8
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
5,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
6,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
8,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [5]:
a = df.as_matrix()

/Users/khuyentran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [6]:
import sys
!{sys.executable} -m pip install pulp

In [7]:
sys.executable


'/Users/khuyentran/opt/anaconda3/bin/python'

In [8]:
from pulp import *
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Ambulance Allocation", LpMaximize)

/Users/khuyentran/opt/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [9]:
from pulp import *

In [10]:
y = LpVariable.dicts("location", list(range(8)), cat='Binary')

In [11]:
x = LpVariable.dicts("reward", list(range(8)), cat='Binary')

In [12]:
y[0]

location_0

In [13]:
#Population of each location
pop = [40, 30, 35, 20, 15, 50, 45, 60]

#There must be at least one ambulance
b = [0,0,0,0,0,0,0,0]

#Define the number of ambulance
n = 1

Objective

In [14]:
prob += lpSum(a[j,i]*pop[i]*y[j] for i in range(8) for j in range(8)) + lpSum(x[i] for i in range(8))

Constraint

In [15]:
for i in range(8):
    prob += lpSum([a[i,j] * y[j]] for j in range(8)) - x[i] >= b[i]

In [16]:
prob += lpSum(y[i] for i in range(8)) == n

In [17]:
prob.solve()

1

In [18]:
for i in range(8):
    print('location {}: {}, penalty {}: {}'.format(i+1,y[i].varValue, i+1, x[i].varValue))

location 1: 0.0, penalty 1: 0.0
location 2: 0.0, penalty 2: 0.0
location 3: 0.0, penalty 3: 0.0
location 4: 0.0, penalty 4: 0.0
location 5: 0.0, penalty 5: 0.0
location 6: 0.0, penalty 6: 1.0
location 7: 0.0, penalty 7: 1.0
location 8: 1.0, penalty 8: 1.0


In [19]:
pulp.value(prob.objective) - sum(x[i].varValue for i in range(8))

155.0

In [20]:
pulp.LpStatus[prob.status]

'Optimal'

In [21]:
prob.writeLP('prob.txt')

[location_0,
 location_1,
 location_2,
 location_3,
 location_4,
 location_5,
 location_6,
 location_7,
 reward_0,
 reward_1,
 reward_2,
 reward_3,
 reward_4,
 reward_5,
 reward_6,
 reward_7]

In [22]:
y = LpVariable.dicts("location", list(range(8)), cat='Binary')
x = LpVariable.dicts("penalty", list(range(8)), cat='Binary')

In [23]:
n1 = 2

In [24]:
prob1 = LpProblem("Ambulance Allocation", LpMaximize)

for i in range (8):
    prob1 += lpSum([a[i,j] * y[j]] for j in range(8)) - x[i] >= b[i]

prob1 += lpSum(y[i] for i in range(8)) == n1

prob1 += lpSum(a[j,i]*pop[i]*y[j] for i in range(8) for j in range(8)) + lpSum(x[i] for i in range(8)) 

prob1.solve()

for i in range(8):
    print('location {}: {}, penalty {}: {}'.format(i+1,y[i].varValue, i+1, x[i].varValue))

location 1: 0.0, penalty 1: 0.0
location 2: 0.0, penalty 2: 0.0
location 3: 0.0, penalty 3: 1.0
location 4: 0.0, penalty 4: 0.0
location 5: 1.0, penalty 5: 1.0
location 6: 0.0, penalty 6: 1.0
location 7: 0.0, penalty 7: 1.0
location 8: 1.0, penalty 8: 1.0


In [25]:
pulp.value(prob1.objective) - sum(x[i].varValue for i in range(8))

300.0

In [26]:
pulp.LpStatus[prob1.status]

'Optimal'

In [27]:
prob1.writeLP('prob1.txt')

[location_0,
 location_1,
 location_2,
 location_3,
 location_4,
 location_5,
 location_6,
 location_7,
 penalty_0,
 penalty_1,
 penalty_2,
 penalty_3,
 penalty_4,
 penalty_5,
 penalty_6,
 penalty_7]

In [28]:
n2 = 3

prob2 = LpProblem("Ambulance Allocation", LpMaximize)

for i in range(8):
    prob2 += lpSum([a[i,j] * y[j] for j in range(8)]) - x[i] >= b[i]

prob2 += lpSum(y[i] for i in range(8)) == n2

prob2 += lpSum(a[j,i]*pop[i]*y[j] for i in range(8) for j in range(8)) + lpSum(x[i] for i in range(8)) 

prob2.solve()

for i in range(8):
    print('location {}: {}, penalty {}: {}'.format(i+1,y[i].varValue, i+1, x[i].varValue))

location 1: 0.0, penalty 1: 0.0
location 2: 0.0, penalty 2: 0.0
location 3: 0.0, penalty 3: 1.0
location 4: 0.0, penalty 4: 1.0
location 5: 1.0, penalty 5: 1.0
location 6: 1.0, penalty 6: 1.0
location 7: 0.0, penalty 7: 1.0
location 8: 1.0, penalty 8: 1.0


In [29]:
pulp.value(prob2.objective) - sum(x[i].varValue for i in range(8))

445.0

In [30]:
pulp.LpStatus[prob2.status]

'Optimal'

In [31]:
prob2.writeLP('prob2.txt')

[location_0,
 location_1,
 location_2,
 location_3,
 location_4,
 location_5,
 location_6,
 location_7,
 penalty_0,
 penalty_1,
 penalty_2,
 penalty_3,
 penalty_4,
 penalty_5,
 penalty_6,
 penalty_7]

In [32]:
n3 = 4

prob3 = LpProblem("Ambulance Allocation", LpMaximize)

for i in range(8):
    prob3 += lpSum([a[i,j] * y[j] for j in range(8)]) - x[i] >= b[i]

prob3 += lpSum(y[i] for i in range(8)) == n3

prob3 += lpSum(a[j,i]*pop[i]*y[j] for i in range(8) for j in range(8)) + lpSum(x[i] for i in range(8)) 

prob3.solve()

for i in range(8):
    print('location {}: {}, penalty {}: {}'.format(i+1,y[i].varValue, i+1, x[i].varValue))

location 1: 0.0, penalty 1: 0.0
location 2: 0.0, penalty 2: 0.0
location 3: 0.0, penalty 3: 1.0
location 4: 0.0, penalty 4: 1.0
location 5: 1.0, penalty 5: 1.0
location 6: 1.0, penalty 6: 1.0
location 7: 1.0, penalty 7: 1.0
location 8: 1.0, penalty 8: 1.0


In [33]:
pulp.value(prob3.objective) - sum(x[i].varValue for i in range(8))

565.0

In [34]:
pulp.LpStatus[prob3.status]

'Optimal'

In [35]:
prob3.writeLP('prob3.txt')

[location_0,
 location_1,
 location_2,
 location_3,
 location_4,
 location_5,
 location_6,
 location_7,
 penalty_0,
 penalty_1,
 penalty_2,
 penalty_3,
 penalty_4,
 penalty_5,
 penalty_6,
 penalty_7]

# 